Income data pulled from the 2012-2016 American Community Survey 5-Year Estimates at https://factfinder.census.gov/ . The search was performed for all 5-digit zip codes in Texas. Any zip code that did not have data for all 6 years were dropped. This removed 17 zip codes from our data set.  

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
from scipy import interpolate



In [44]:
def interp1d(xnew, x, y, kind):
    f = interpolate.interp1d(x, y, kind="linear")
#     print(x, xnew)
    return f(xnew)

In [45]:
income_11_df = pd.read_csv("household_income_11.csv", skiprows=1)
income_11_df = income_11_df[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_11_df = income_11_df.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_11_df = income_11_df[income_11_df["Median Income (dollars)"] != '-']
income_11_df["Year"] = 2011

income_12_df = pd.read_csv("household_income_12.csv", skiprows=1)
income_12_df = income_12_df[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_12_df = income_12_df.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_12_df = income_12_df[income_12_df["Median Income (dollars)"] != '-']
income_12_df["Year"] = 2012

income_13_df = pd.read_csv("household_income_13.csv", skiprows=1)
income_13_df = income_13_df[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_13_df = income_13_df.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_13_df = income_13_df[income_13_df["Median Income (dollars)"] != '-']
income_13_df["Year"] = 2013

income_14_df = pd.read_csv("household_income_14.csv", skiprows=1)
income_14_df = income_14_df[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_14_df = income_14_df.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_14_df = income_14_df[income_14_df["Median Income (dollars)"] != '-']
income_14_df["Year"] = 2014

income_15_df = pd.read_csv("household_income_15.csv", skiprows=1)
income_15_df = income_15_df[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_15_df = income_15_df.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_15_df = income_15_df[income_15_df["Median Income (dollars)"] != '-']
income_15_df["Year"] = 2015

income_16_df = pd.read_csv("household_income_16.csv", skiprows=1)
income_16_df = income_16_df[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_16_df = income_16_df.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_16_df = income_16_df[income_16_df["Median Income (dollars)"] != '-']
income_16_df["Year"] = 2016

income_df = income_11_df
income_df = income_df.append(income_12_df, ignore_index=True)
income_df = income_df.append(income_13_df, ignore_index=True)
income_df = income_df.append(income_14_df, ignore_index=True)
income_df = income_df.append(income_15_df, ignore_index=True)
income_df = income_df.append(income_16_df, ignore_index=True)
income_df.reset_index(inplace=True)

In [46]:
# perform data cleansing
income_df["Number of Households"] = income_df["Number of Households"].apply(lambda x: int(x))
income_df["Median Income (dollars)"].loc[income_df["Median Income (dollars)"] == '2,500-'] = 25000
income_df["Median Income (dollars)"].loc[income_df["Median Income (dollars)"] == '250,000+'] = 25000
income_df = income_df.loc[income_df["Median Income (dollars)"] != "(X)"]
income_df["Median Income (dollars)"] = income_df["Median Income (dollars)"].apply(pd.to_numeric)
income_df["Year"] = income_df["Year"].apply(pd.to_numeric)

/home/aidin/miniconda3/envs/08-project-one/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [57]:
# perform interplation to fill month values
interp_kind = "slinear"
income_interp_df = pd.DataFrame()
for zipcode in income_df["Zip Code"].unique():
    # get zip code data frame
    zipcode_df = income_df.loc[income_df["Zip Code"] == zipcode, :]
    
    # get values
    n_years = zipcode_df.shape[0]
    years = zipcode_df["Year"].values
    incomes = zipcode_df["Median Income (dollars)"].values
    households = zipcode_df["Number of Households"].values
    
    # get interpolations
    x_interp = np.linspace(1, n_years * 12,  n_years * 12)
    x = np.insert(np.linspace(1, n_years,  n_years) * 12, 0, 0)
    
    incomes_interp = interp1d(
        x_interp, x, np.insert(incomes, 0, incomes[0]), kind=interp_kind).astype(float).tolist()
    households_interp = interp1d(
        x_interp, x, np.insert(households, 0, households[0]), kind=interp_kind).astype(int).tolist()
    years_interp = np.repeat(years, 12).tolist()
    month_interp = np.tile(np.linspace(1, 12, 12).astype(int), n_years).tolist()
    
    
    # create new
    zipcode_interp_df = pd.DataFrame({"Year": years_interp,
                                      "Month": month_interp,
                                      "Number of Households": households_interp,
                                      "Median Income (dollars)": incomes_interp})
    income_interp_df = income_interp_df.append(zipcode_interp_df)

income_interp_df

,Year,Month,Number of Households,Median Income (dollars)
0,2011,1,605,42009.00
1,2011,2,605,42009.00
2,2011,3,605,42009.00
3,2011,4,605,42009.00
4,2011,5,605,42009.00
5,2011,6,605,42009.00
6,2011,7,605,42009.00
7,2011,8,605,42009.00
8,2011,9,605,42009.00
9,2011,10,605,42009.00
